# CS-E-106: Data Modeling
## Fall 2019: Lab 02

In [36]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import io
import requests
from sklearn.metrics import r2_score

**Example 1 (the one from CH. 2):** Very similar to HW1

**(Textbook 2.62) Refer to the CDI data set in Appendix C.2 and Project l.43.**
This data set provides selected county demographic information (CDI) for 440 of the most populous counties in the United States. Each line of the data set has an identification number with a county name and state abbreviation and provides information on 14 variables for a single county. Counties with missing data were deleted from the data set. The information generally pertains to the years 1990 and 1992... More information on page 1349.}

In [27]:
url="https://www.stat.purdue.edu/~bacraig/datasets525/APPENC02.txt"
s=requests.get(url).content

cdiColNames =  ["id", "county", "state", "landArea", "totPop", 
                 "percAge18_34", "percAge65plus", "actPhysicians",
                 "hospBeds", "totSerCrimes", "percHSgrads", "percBachDeg",
                 "percBelowPov", "percUnempl", "perCapitaInc", 
                 "totPersIncome", "geoRegion"]

df_cdi = pd.read_fwf(io.StringIO(s.decode('utf-8')), sep=" ", names=cdiColNames)

# df_cdi. = cdiColNames

# Displays 6 rows
display(df_cdi.head(10))

# Numeric summaries
df_cdi.describe()

,id,county,state,landArea,totPop,percAge18_34,percAge65plus,actPhysicians,hospBeds,totSerCrimes,percHSgrads,percBachDeg,percBelowPov,percUnempl,perCapitaInc,totPersIncome,geoRegion
0,1,Los_Angeles,CA,4060,8863164,32.1,9.7,23677,27700,688936,70.0,22.3,11.6,8.0,20786,184230,4
1,2,Cook,IL,946,5105067,29.2,12.4,15153,21550,436936,73.4,22.8,11.1,7.2,21729,110928,2
2,3,Harris,TX,1729,2818199,31.3,7.1,7553,12449,253526,74.9,25.4,12.5,5.7,19517,55003,3
3,4,San_Diego,CA,4205,2498016,33.5,10.9,5905,6179,173821,81.9,25.3,8.1,6.1,19588,48931,4
4,5,Orange,CA,790,2410556,32.6,9.2,6062,6369,144524,81.2,27.8,5.2,4.8,24400,58818,4
5,6,Kings,NY,71,2300664,28.3,12.4,4861,8942,680966,63.7,16.6,19.5,9.5,16803,38658,1
6,7,Maricopa,AZ,9204,2122101,29.2,12.5,4320,6104,177593,81.5,22.1,8.8,4.9,18042,38287,4
7,8,Wayne,MI,614,2111687,27.4,12.5,3823,9490,193978,70.0,13.7,16.9,10.0,17461,36872,2
8,9,Dade,FL,1945,1937094,27.1,13.9,6274,8840,244725,65.0,18.8,14.2,8.7,17823,34525,3
9,10,Dallas,TX,880,1852810,32.6,8.2,4718,6934,214258,77.1,26.3,10.4,6.1,21001,38911,3


,id,landArea,totPop,percAge18_34,percAge65plus,actPhysicians,hospBeds,totSerCrimes,percHSgrads,percBachDeg,percBelowPov,percUnempl,perCapitaInc,totPersIncome,geoRegion
count,440.000000,440.000000,4.400000e+02,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,220.500000,1041.411364,3.930109e+05,28.568409,12.169773,987.997727,1458.627273,27111.618182,77.560682,21.081136,8.720682,6.596591,18561.481818,7869.272727,2.461364
std,127.161315,1549.922081,6.019870e+05,4.191083,3.992666,1789.749540,2289.134128,58237.506385,7.015159,7.654524,4.656737,2.337924,4059.192009,12884.321463,1.033982
min,1.000000,15.000000,1.000430e+05,16.400000,3.000000,39.000000,92.000000,563.000000,46.600000,8.100000,1.400000,2.200000,8899.000000,1141.000000,1.000000
25%,110.750000,451.250000,1.390272e+05,26.200000,9.875000,182.750000,390.750000,6219.500000,73.875000,15.275000,5.300000,5.100000,16118.250000,2311.000000,2.000000
50%,220.500000,656.500000,2.172805e+05,28.100000,11.750000,401.000000,755.000000,11820.500000,77.700000,19.700000,7.900000,6.200000,17759.000000,3857.000000,3.000000
75%,330.250000,946.750000,4.360645e+05,30.025000,13.625000,1036.000000,1575.750000,26279.500000,82.400000,25.325000,10.900000,7.500000,20270.000000,8654.250000,3.000000
max,440.000000,20062.000000,8.863164e+06,49.700000,33.800000,23677.000000,27700.000000,688936.000000,92.900000,52.300000,36.300000,21.300000,37541.000000,184230.000000,4.000000


In [87]:
def get_rsq(response, predictor, df):
    X = sm.add_constant(df[predictor])
    lm_fit = sm.OLS(df[response], X).fit()
    return(lm_fit.rsquared)

In [88]:
dict_rsq = {}
variables = []
rsq = []
for var in cdiColNames[4:16]:
    if var != "actPhysicians":
        variables.append(var)
        rsq.append(get_rsq("actPhysicians", var, df=df_cdi))

dict_rsq = {"Variable": variables, "R-Sq": rsq}

print(dict_rsq)
df_rsq = pd.DataFrame(dict_rsq)

{'Variable': ['totPop', 'percAge18_34', 'percAge65plus', 'hospBeds', 'totSerCrimes', 'percHSgrads', 'percBachDeg', 'percBelowPov', 'percUnempl', 'perCapitaInc', 'totPersIncome'], 'R-Sq': [0.8840674122496884, 0.014327908116359978, 9.788322648196512e-06, 0.9033825654973335, 0.6731537526630951, 1.8046222736156636e-05, 0.056057885859429946, 0.004113459125813956, 0.00255187801271739, 0.09994110082218732, 0.8989136554632065]}


/home/srk-apts/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [89]:
df_rsq

,Variable,R-Sq
0,totPop,0.884067
1,percAge18_34,0.014328
2,percAge65plus,0.000010
3,hospBeds,0.903383
4,totSerCrimes,0.673154
5,percHSgrads,0.000018
6,percBachDeg,0.056058
7,percBelowPov,0.004113
8,percUnempl,0.002552
9,perCapitaInc,0.099941
